# Statistics

In [ ]:
!pip install -U "datasets<3.0" #Con el fin de poder cargar el dataset de ejemplo

In [ ]:
import datasets  # Biblioteca de manejo de conjuntos de datos para procesamiento de lenguaje natural
import re  # Biblioteca de expresiones regulares
import collections  # Módulo de tipos de datos adicionales de Python
import itertools  # Módulo de herramientas de manejo de iterables
import matplotlib.pyplot as plt  # Biblioteca de visualización
import wordcloud  # Biblioteca de visualización de nubes de palabras
import PIL  # Biblioteca de manejo de imágenes
import numpy as np  # Biblioteca de manejo de datos vectoriales
import spacy.lang.es # Biblioteca de procesamiento de lenguaje natural en español

Cargaremos y analizaremos nuestro conjunto de datos y lo compararemos con otro de opiniones de películas.

In [ ]:
# Cargamos las particiones de entrenamiento y prueba
spanish_diagnostics = datasets.load_dataset(
    'fvillena/spanish_diagnostics', split="train+test")

In [ ]:
spanish_diagnostics

Esta es una de las instancias de nuestro conjunto de datos.

In [ ]:
spanish_diagnostics[2]["text"]

Normalizaremos nuestro texto con la función que programamos la clase anterior y tokenizaremos el texto mediante espacio.

In [ ]:
#Definimos una función para normalizar texto
def normalize(text, remove_tildes = True):
    """Normaliza una cadena de texto convirtiéndo todo a minúsculas, quitando los caracteres no alfabéticos y los tildes"""
    text = text.lower() # Llevamos todo a minúscula
    text = re.sub(r'[^A-Za-zñáéíóú]', ' ', text) # Reemplazamos los caracteres no alfabéticos por un espacio
    if remove_tildes:
        text = re.sub('á', 'a', text) # Reemplazamos los tildes
        text = re.sub('é', 'e', text)
        text = re.sub('í', 'i', text)
        text = re.sub('ó', 'o', text)
        text = re.sub('ú', 'u', text)
    return text

In [ ]:
spanish_diagnostics_normalized = spanish_diagnostics.map(
    lambda x: {
        "normalized_text": normalize(x["text"])
    })

In [ ]:
spanish_diagnostics_normalized_tokenized = spanish_diagnostics_normalized.map(
    lambda x: {
        "tokenized_text": x["normalized_text"].split()
    })

In [ ]:
spanish_diagnostics_normalized_tokenized

In [ ]:
spanish_diagnostics_normalized_tokenized["tokenized_text"][:3]

El conjunto de datos con el cual nos compararemos es un corpus de chistes en español.

In [ ]:
# Cargamos el dataset de Muchocine
chistes = datasets.load_dataset('mrm8488/CHISTES_spanish_jokes', split="train")

In [ ]:
chistes

In [ ]:
chistes_normalized = chistes.map(
    lambda x: {  # Utilizamos una función anónima que devuelve un diccionario
        # Esta es una nueva característica que agregaremos a nuestro conjunto de datos.
        "normalized_text": normalize(x["text"])
    })

In [ ]:
chistes_normalized_tokenized = chistes_normalized.map(
    lambda x: {
        "tokenized_text": x["normalized_text"].split()
    })

In [ ]:
chistes_normalized_tokenized["tokenized_text"][0][:10]

Un *corpus* (plural *corpora*) es el nombre que reciben los conjuntos de datos de texto, estos *corpora* se componen de documentos, los cuales son cada una de las instancias de texto de nuestro *corpus*. Una de las métricas de descripción de *corpora* es el número de documentos, o sea, la cantidad de instancias que tiene mi conjunto de datos.

El número de documentos del *corpus* spanish_diagnostics es mucho mayor al *corpus* muchocine

In [ ]:
spanish_diagnostics_n_documents = len(spanish_diagnostics)
spanish_diagnostics_n_documents  # Número de documentos

In [ ]:
chistes_n_documents = len(chistes)
chistes_n_documents

A nivel de tokens en nuestro corpus se pueden describir principalmente el número de tokens, el cual es la cantidad total de tokens que presenta nuesto corpus y el tamaño del voculario, el cual es la cantidad de tokens distintos que tiene nuestro corpus.

Utilizaremos la clase collections.Counter para contar las apariciones de cada una de las palabras del vocabulario en los corpora.

In [ ]:
spanish_diagnostics_words = collections.Counter(  # Con esta clase contamos cada una de las apariciones de las palabras
    itertools.chain(  # Con esta función aplanamos nuestra lista anidada de tokens
        *spanish_diagnostics_normalized_tokenized["tokenized_text"]
    )
)

In [ ]:
chistes_words = collections.Counter(itertools.chain(
    *chistes_normalized_tokenized["tokenized_text"]))

In [ ]:
spanish_diagnostics_n_tokens = sum(
    [count for word, count in spanish_diagnostics_words.items()])
spanish_diagnostics_n_tokens

In [ ]:
chistes_n_tokens = sum([count for word, count in chistes_words.items()])
chistes_n_tokens

In [ ]:
spanish_diagnostics_vocabulary_size = len(spanish_diagnostics_words)
spanish_diagnostics_vocabulary_size

In [ ]:
chistes_vocabulary_size = len(chistes_words)
chistes_vocabulary_size

La diversidad léxica es una relación entre el tamaño del vocabulario y la cantidad de tokens. Una diversidad léxica mayor denota una riqueza léxica.

El corpus spanish_diagnostics presenta una diversidad léxica mayor que el corpus muchocine.

In [ ]:
spanish_diagnostics_lexical_diversity = spanish_diagnostics_vocabulary_size / \
    spanish_diagnostics_n_tokens
spanish_diagnostics_lexical_diversity

In [ ]:
chistes_lexical_diversity = chistes_vocabulary_size / chistes_n_tokens
chistes_lexical_diversity

Exploraremos las palabras ordenadas por frecuencia de aparición. Se sabe que los lenguajes naturales obedecen a la ley de Zipf, la cual define que la frecuencia de cualquier palabra es inversamente proporcional a su posición en una tabla de frecuencias.

Observamos que a medida que vamos avanzando en la posición de frecuencias de las palabras, la frecuencia de cada una cae muy rapidamente.

In [ ]:
# Con método collections.Counter.most_common() podemos extraer las palabras más frecuentes del corpus
spanish_diagnostics_words_top_words = spanish_diagnostics_words.most_common(30)

In [ ]:
chistes_words_top_words = chistes_words.most_common(30)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(10, 8))
axs[0].barh(
    [word for word, f in spanish_diagnostics_words_top_words],
    [f for word, f in spanish_diagnostics_words_top_words],
)
axs[1].barh(
    [word for word, f in chistes_words_top_words],
    [f for word, f in chistes_words_top_words]
)
axs[0].invert_yaxis()
axs[1].invert_yaxis()
axs[0].set_title("spanish_diagnostics")
axs[1].set_title("chistes")
axs[0].set_ylabel("Word")
axs[0].set_xlabel("Frequency")
axs[1].set_xlabel("Frequency")
plt.show()

Para poder visualizar de mejor manera esta característica podemos visualizar las frecuencias con un gráfico de línea sobre unos ejes en escala logarítmica.

In [ ]:
plt.plot([count for word, count in spanish_diagnostics_words.most_common()],
         label="spanish_diagnostics")
plt.plot([count for word, count in chistes_words.most_common()], label="chistes")
plt.yscale("log", base=2)
plt.xscale("log", base=2)
plt.ylabel("Frequency")
plt.xlabel("Rank")
plt.legend()
plt.show()

Una de las visualizaciones de corpora más utilizadas es el Word CLoud, la cual es una representación de la frecuencia (u otra métrica asociada) de las palabras mediante el tamaño de las mismas sobre un lienzo.

Instanciamos un objeto de la clase wordcloud.WordCloud con los parámetros de la visualización.

In [ ]:
wc = wordcloud.WordCloud(  # Objeto de WordCloud
    background_color="white",  # El fondo de la visualización es blanco
    colormap="Blues"  # Las palabras tendrán color azul
)

Nos podemos dar cuenta que hay palabras que tienen un tamaño muy grande (o frecuencia muy alta) pero que no nos ayudan a entender el contenido de los corpora. A estas palabras se les denomina stopwords y típicamente se las elimina para poder realizar nuestro análisis.

In [ ]:
fig, axs = plt.subplots(nrows=2, figsize=(10, 10))
# Creamos el wordcloud desde las frecuencias de las palabras
axs[0].imshow(wc.generate_from_frequencies(
    spanish_diagnostics_words), interpolation="bilinear")
axs[1].imshow(wc.generate_from_frequencies(
    chistes_words), interpolation="bilinear")
axs[0].axis("off")
axs[1].axis("off")
axs[0].set_title("spanish_diagnostics")
axs[1].set_title("chistes")
plt.show()

Existen listas de stopwords típicas en algunas bibliotecas de procesamiento de lenguaje natural.

In [ ]:
# La biblioteca Spacy tiene una lista de stopwords en español
stopwords = spacy.lang.es.stop_words.STOP_WORDS

In [ ]:
fig, axs = plt.subplots(nrows=2, figsize=(10, 10))
axs[0].imshow(wc.generate_from_frequencies(
    {word: f for word, f in spanish_diagnostics_words.items(
    ) if not word in stopwords}  # Acá eliminamos las stopwords
), interpolation="bilinear")
axs[1].imshow(wc.generate_from_frequencies(
    {word: f for word, f in chistes_words.items() if not word in stopwords}
), interpolation="bilinear")
axs[0].axis("off")
axs[1].axis("off")
axs[0].set_title("spanish_diagnostics")
axs[1].set_title("chistes")
plt.show()

Si bien una lista de stopwords es normalemnte suficiente para limpiar el texto también podemos extender nuestra lista con palabras que nosotros consideremos.

In [ ]:
stopwords_extended = list(stopwords) + [  # Agregamos estas palabras que nos están ensuciando la visualización
    "y",
    "a",
    "o"
]

Ahora con nuestros corpora limpios podemos visualizar fácilmente el contenido de nuestros corpora. En el corpus spanish_diagnostics claramente vemos que las palabras más importantes en la visualización son las relacionadas a salud y en el corpus muchocine son palabras relacionadas al cine.

In [ ]:
fig, axs = plt.subplots(nrows=2, figsize=(10, 10))
axs[0].imshow(wc.generate_from_frequencies(
    {word: f for word, f in spanish_diagnostics_words.items()
     if not word in stopwords_extended}
), interpolation="bilinear")
axs[1].imshow(wc.generate_from_frequencies(
    {word: f for word, f in chistes_words.items() if not word in stopwords_extended}
), interpolation="bilinear")
axs[0].axis("off")
axs[1].axis("off")
axs[0].set_title("spanish_diagnostics")
axs[1].set_title("chistes")
plt.show()

Para poder enriquecer nuestra visualización podemos agregar máscaras a nuestras nubes y así comunicar mejor la información.

In [ ]:
# Importamos imágenes que utilizaremos como máscaras
# Este acceso funciona cargando las imágenes directamente desde la sección actual en Colab,
# aunque también se pueden cargar desde Drive.
# O bien, si las imágenes se encuentran en la misma carpeta (de forma local)
import requests # Importamos la librería requests para descargar imágenes desde URLs
from io import BytesIO # Importamos BytesIO para manejar los datos de la imagen en memoria

red_cross_link = "https://raw.githubusercontent.com/dccuchile/CC66Q/main/tutorials/data/red_cross.png"
play_pause_link = "https://raw.githubusercontent.com/dccuchile/CC66Q/main/tutorials/data/play_pause.png"

# Descargamos las imágenes y las abrimos con PIL
response_red_cross = requests.get(red_cross_link)
red_cross_image = PIL.Image.open(BytesIO(response_red_cross.content))

response_play_pause = requests.get(play_pause_link)
play_pause_image = PIL.Image.open(BytesIO(response_play_pause.content))

In [ ]:
fig, axs = plt.subplots(nrows=2, figsize=(10, 5))

axs[0].imshow(red_cross_image)
axs[0].axis("off")

axs[1].imshow(play_pause_image)
axs[1].axis("off")

plt.show()

In [ ]:
# Convertimos las imágenes a máscaras en formato de array
red_cross = np.array(red_cross_image)
play_pause = np.array(play_pause_image)

# Instanciamos un objeto por cada uno de nuestros corpora
wc_spanish_diagnostics = wordcloud.WordCloud(
    background_color="white",
    colormap="Reds",
    mask=red_cross,
    contour_width=3,
    contour_color='red'
)
wc_chistes = wordcloud.WordCloud(
    background_color="white",
    colormap="Blues",
    mask=play_pause,
    contour_width=3,
    contour_color='blue'
)

In [ ]:
fig, axs = plt.subplots(nrows=2, figsize=(10, 10))
axs[0].imshow(wc_spanish_diagnostics.generate_from_frequencies(
    {word: f for word, f in spanish_diagnostics_words.items()
     if not word in stopwords_extended}
), interpolation="bilinear")
axs[1].imshow(wc_chistes.generate_from_frequencies(
    {word: f for word, f in chistes_words.items() if not word in stopwords_extended}
), interpolation="bilinear")
axs[0].axis("off")
axs[1].axis("off")
axs[0].set_title("spanish_diagnostics")
axs[1].set_title("chistes")
plt.show()